# Multi-variable Linear Regression

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
import numpy as np
import matplotlib.pyplot as plt

tf.__version__

# Hypothesis

## $$ H(x_1, x_2, x_3) = \underline{w_1 x_1 + w_2 x_2 + w_3 x_3} + b $$

## $$ = w_1 x_1 + w_2 x_2 + w_3 x_3 $$ 

## $$ =  \begin{pmatrix} w_{ 1 } & w_{ 2 } & w_{ 3 } \end{pmatrix}\cdot \begin{pmatrix} x_{ 1 } \\ x_{ 2 } \\ x_{ 3 } \end{pmatrix} $$

## $$ H(x_1, x_2, x_3) = WX $$ 



In [ ]:
x_data = [
    [1., 0., 3., 0., 5.],
    [0., 2., 0., 4., 0.]
]
y_data  = [1, 2, 3, 4, 5]

W = tf.Variable(tf.random.normal([1, 2], -1.0, 1.0))
b = tf.Variable(tf.random.normal([1], -1.0, 1.0))

learning_rate = tf.Variable(0.001)

for i in range(100):
   with tf.GradientTape() as tape:
    hypothesis = tf.matmul(W, x_data) + b # (1, 2) * (2, 5) = (1, 5)
    cost = tf.reduce_mean(tf.square(hypothesis - y_data))

    W_grad, b_grad = tape.gradient(cost, [W, b])
    W.assign_sub(learning_rate * W_grad)
    b.assign_sub(learning_rate * b_grad)
    
    if i % 10 == 0:
        print("step: {:3} \t cost: {:5.4f} \t w[0][0]: {:5.4f} \t w[0][1]: {:5.4f} \t b: {:5.4f}".format(
            i, cost.numpy(), W.numpy()[0][0], W.numpy()[0][1], b.numpy()[0]))

## Test Score


x1 | x2 | x3 | Y
---- | ---- | ----| ----
96 | 91 | 99 | 194
88 | 85 | 82 | 181
78 | 77 | 73 | 177
67 | 66 | 61 | 164
55 | 51 | 53 | 157

# Hypothesis using matrix

$$ [5, 3] \cdot [?, ?] = [5, 1] $$

$$ H(X) = XW + b $$


In [ ]:
data = np.array([
    # X1,   X2,    X3,   y
    [ 96.,  91.,  99., 194. ],
    [ 88.,  85.,  82., 181. ],
    [ 78.,  77.,  73., 177. ],
    [ 67.,  66., 61., 164. ],
    [ 55.,  51.,  53., 157. ]
], dtype=np.float32)

# slice data
X = data[:, :-1]
y = data[:, [-1]]

W = tf.Variable(tf.random.normal([3, 1]))
b = tf.Variable(tf.random.normal([1]))

learning_rate = 0.000001

def predict(X):
    return tf.matmul(X, W) + b

print("epoch | cost")

n_epochs = 1000
for i in range(n_epochs):
    # tf.GradientTape() to record the gradient of the cost function
    with tf.GradientTape() as tape:
        cost = tf.reduce_mean((tf.square(predict(X) - y)))

    # Loss함수의 gradient를 계산한다.
    W_grad, b_grad = tape.gradient(cost, [W, b])

    # 파라미터 업데이트 (W and b)
    W.assign_sub(learning_rate * W_grad)
    b.assign_sub(learning_rate * b_grad)
    
    if i % 100 == 0:
        print("{:5} | {:10.4f}".format(i, cost.numpy()))

## 데이터를 기반으로예측해보자

In [ ]:
def predict(X):
    return tf.matmul(X, W) + b # 위쪽에 선언되어 있다.

predict(X).numpy() # prediction, 예측값

In [ ]:
# 새로운 데이터에 대한 예측

predict([[ 89.,  95.,  92.],[ 84.,  92.,  85.]]).numpy() 

### with Tensorflow

In [ ]:
data = np.array([
    # X1,   X2,    X3,   y
    [ 96.,  91.,  99., 194. ],
    [ 88.,  85.,  82., 181. ],
    [ 78.,  77.,  73., 177. ],
    [ 67.,  66., 61., 164. ],
    [ 55.,  51.,  53., 157. ]
], dtype=np.float32)

# slice data
X = data[:, :-1]
y = data[:, [-1]]

dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.batch(batch_size=1)

model = Sequential([
    layers.Dense(1)
])

model.compile(optimizer='adam',
              loss='mae',
              metrics=['mae'])

model.fit(dataset, epochs=1000)

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
test_loss, test_mae = model.evaluate(X, y, verbose=0)
print('Test MSE:', test_mae) 

In [ ]:
for x, y in dataset:
    print(x)
    print(y)
    print(model(x))
    break